In [1]:
## generate.ipynb
## Code to generate a Gaussian Random Field for velocities as a standard model for cloud
## scale star formation simulations. The field is guaranteed to be solenoidal.

import numpy as np
import h5py

In [2]:
def init_velocity_field(sigma, kspec, kmin, kmax, N, seed=42):
    """
    Initializes a 3D, periodic gaussian random field with a given power spectrum

    Parameters
    ----------
    kspec : float
        The spectral index of the power spectrum
    sigma : float
        The standard deviation of the velocity field
    kmin : float
        The minimum wavenumber for the power spectrum
    kmax : float
        The maximum wavenumber for the power spectrum
    N : int
        The size of the grid (N x N x N)
    """
    kmin *= 2*np.pi
    kmax *= 2*np.pi
    kx = np.fft.fftfreq(N, d=1./N) * 2*np.pi
    ky = np.fft.fftfreq(N, d=1./N) * 2*np.pi
    kz = np.fft.fftfreq(N, d=1./N) * 2*np.pi
    KS = np.meshgrid(kx, ky, kz, indexing="ij")
    (KX, KY, KZ) = KS
    K = np.sqrt(KX**2 + KY**2 + KZ**2)
    mask = K > 0
    mask1 = K < kmin
    mask2 = K > kmax

    VK = []
    for i in range(3):
        np.random.seed(seed + i)
        vk = np.zeros((N,N,N), dtype=complex)
        phase = np.fft.fftn(np.random.normal(size=(N,N,N)))
        vk[mask] = (K[mask]**(-1*kspec))*phase[mask]
        vk[mask1] = 0
        vk[mask2] *= np.exp(-1*(K[mask2]/kmax)**2)*np.e
        VK.append(vk)
    VK = np.array(VK)

    VKP = np.zeros_like(VK, dtype=complex)
    for i in range(3):
        for j in range(3):
            if i == j:
                VKP[i] += VK[j]
            VKP[i][mask] -= (KS[i]*KS[j]*VK[j])[mask]/(K[mask]**2)

    (vx, vy, vz) = [np.fft.ifftn(vk).real for vk in VKP]
    sigma_res = np.sqrt(np.std(vx)**2 + np.std(vy)**2 + np.std(vz)**2)

    vx *= sigma/sigma_res
    vy *= sigma/sigma_res
    vz *= sigma/sigma_res
    return (vx,vy,vz)

In [3]:
# Example usage
N = 128
sigma = 1.0
kspec = 2
kmin = 4
kmax = 32
# Lorentz's Birthday: 18th of July, 1853
seed = 18071853


(vx,vy,vz) = init_velocity_field(sigma, kspec, kmin, kmax, N, seed=seed)

In [4]:
# Use H5py to create a HDF5 file that stores the velocity field information
f = h5py.File("velocity_field.h5", "w")
f.create_dataset("vx", data=vx)
f.create_dataset("vy", data=vy)
f.create_dataset("vz", data=vz)
f.close()

In [5]:
print("Velocity field initialized with shape:", vx.shape)
print("vx mean:", np.mean(vx), "std:", np.std(vx))
print("vy mean:", np.mean(vy), "std:", np.std(vy))
print("vz mean:", np.mean(vz), "std:", np.std(vz))

Velocity field initialized with shape: (128, 128, 128)
vx mean: 2.981555974335137e-19 std: 0.5742885300858656
vy mean: -1.5449880957918438e-18 std: 0.5778606447860277
vz mean: -1.870248747537495e-18 std: 0.5798877127679908


In [6]:
## How to read the velocity field back in
with h5py.File("velocity_field.h5", "r") as f:
    vx = f["vx"][:]
    vy = f["vy"][:]
    vz = f["vz"][:]
f.close()

In [7]:
vz.shape

(128, 128, 128)